# Create functions and initialize libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
# Store results of API call
import json
from datetime import date
import numpy as np
fips = pd.read_csv('fips_codes.csv', delimiter=',', dtype = str)
fips = fips.sort_values(by = 'Name')
API_key = 'YOUR CENSUS BUREAU KEY GOES HERE'

def LinkedIn_Monster_Job_Table(States, JobTitle, verbose = False):
    df_columns = ['Title', 'Company', 'Link', 'City', 'State', 'Days Listed', 'Source']
    final_df = pd.DataFrame(columns = df_columns)
    
    for State in States:
        MonJob = JobTitle.replace(" ", "-")
        LInJob = JobTitle.replace(" ", "%20")
        IndeedJob = JobTitle.replace(" ", "+")
        if verbose == True:
            print(f'Creating Jobs Table For {State}')

        # Monster.com
        URL = f'https://www.monster.com/jobs/search/?q={MonJob}&where={State}'
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        results = soup.find(id='ResultsContainer')
        job_elems = results.find_all('section', class_='card-content')
        df = pd.DataFrame(columns = df_columns)

        for job_elem in job_elems:
            title_elem = job_elem.find('h2', class_='title')
            company_elem = job_elem.find('div', class_='company')
            location_elem = job_elem.find('div', class_='location')
            link_elem = job_elem.find('a')
            time = job_elem.find('time')
            if None in (title_elem, company_elem, location_elem, time):
                continue
            #if verbose == True:
                #print(time.text.strip())
            try:
                time_mon = int(re.search(r'\d+', time.text.strip()).group(0))
            except:
                time_mon = 0
            df = df.append({'Title': title_elem.text.strip(),
                            'Company': company_elem.text.strip(),
                            'Link': link_elem['href'],
                            'City':location_elem.text.split(', ')[0].strip(),
                            'State':State,
                            'Days Listed': time_mon},
                           ignore_index=True)
        df['Source'] = 'Monster'
        
        if verbose == True:
            print(f'    Finished Monster Table For {State}')

            
            
        # LinkedIn.com
        URL_2 = f'https://www.linkedin.com/jobs/search/?keywords={LInJob}&location={State}%2C%20United%20States'
        page_2 = requests.get(URL_2)
        soup_2 = BeautifulSoup(page_2.content, 'html.parser')
        results_2 = soup_2.find_all('li', class_='result-card')
        df_2 = pd.DataFrame(columns = df_columns)

        for job_elem in results_2:
            title_elem = job_elem.find('h3', class_='job-result-card__title')
            company_elem = job_elem.find('a', class_='job-result-card__subtitle-link')
            location_elem = job_elem.find('span', class_='job-result-card__location')
            link_elem = job_elem.find('a')
            time = job_elem.find('time')
            if None in (title_elem, company_elem, location_elem, time):
                continue
            df_2 = df_2.append({'Title': title_elem.text.strip(),
                                'Company': company_elem.text.strip(),
                                'Link': link_elem['href'],
                                'City':location_elem.text.split(', ')[0].strip(),
                                'State':State,
                                'Days Listed': time.text.strip()},
                               ignore_index=True)
        df_2['Source'] = 'LinkedIn'

        info_dict = {}
        for info in df_2['Days Listed'].unique():
            info_break = info.split()
            if info_break[1] in ['minute', 'minutes']:
                info_dict.update([(info, (1/1440)*int(info_break[0]))])
            if info_break[1] in ['hour', 'hours']:
                info_dict.update([(info, (1/24)*int(info_break[0]))])
            elif info_break[1] in ['day', 'days']:
                info_dict.update([(info, 1*int(info_break[0]))])
            elif info_break[1] in ['week', 'weeks']:
                info_dict.update([(info, 7*int(info_break[0]))])
            elif info_break[1] in ['month', 'months']:
                info_dict.update([(info, 30*int(info_break[0]))])
            elif info_break[1] in ['year', 'years']:
                info_dict.update([(info, 365*int(info_break[0]))])

        df_2 = df_2.replace({'Days Listed': info_dict})
        
        if verbose == True:
            print(f'    Finished LinkedIn Table For {State}')
        
        
        
        # Indeed.com    
        URL_3 = f'https://www.indeed.com/jobs?q={IndeedJob}&l={State}'
        page_3 = requests.get(URL_3)
        soup_3 = BeautifulSoup(page_3.content, 'html.parser')
        results_3 = soup_3.find_all('div', class_='jobsearch-SerpJobCard')
        df_3 = pd.DataFrame(columns = df_columns)
        
        for job_elem in results_3:
            title_elem = job_elem.find('h2', class_='title')
            company_elem = job_elem.find('span', class_='company')
            location_elem = job_elem.find('span', class_='location')
            link_elem = job_elem.find('a')
            time = job_elem.find('span', class_='date')
            if None in (title_elem, company_elem, location_elem, time):
                continue
            try:
                time_mon = int(re.search(r'\d+', time.text.strip()).group(0))
            except:
                time_mon = 0
            df_3 = df_3.append({'Title': title_elem.text.strip(),
                                'Company': company_elem.text.strip(),
                                'Link': 'https://www.indeed.com' + link_elem['href'],
                                'City':location_elem.text.split(', ')[0].strip(),
                                'State':State,
                                'Days Listed': time_mon},
                               ignore_index=True)
        df_3['Source'] = 'Indeed'
        
        #print(df_3)
        
        if verbose == True:
            print(f'    Finished Indeed Table For {State}')

        final_df = pd.concat([final_df, df, df_2, df_3], ignore_index = True, sort = False).sort_values(by = 'Company', ascending = True)
        if verbose == True:
            print(f'Finished creating Jobs Table For {State} \n')
    if verbose == True:
            print(f'Combined Jobs Tables For {States} \n')
    return final_df

In [2]:
def Census_Place_Jobs_Table(variableCodes, 
                            variableNames, 
                            states, 
                            replace_dict, 
                            job_search,
                            year = 2018,
                            verbose = False,
                            expand = False):
    if verbose == True:
        print(f'Creating Pops Table For {states}')
    states.sort()
    fips = pd.read_csv('fips_codes.csv', delimiter=',', dtype = str)
    fips = fips.sort_values(by = 'Name')
    state_code = fips['Fips'][fips.Name.isin(states)].to_string(header=False,index=False).split('\n')
    state_code = ','.join(state_code).replace(' ','')
    apiResponse = requests.get(f'https://api.census.gov/data/{year}/acs/acs5?get=NAME,{variableCodes.replace(" ", "")}&for=place:*&in=state:{state_code.replace(" ", "")}&key={API_key}')
    formattedResponse = json.loads(apiResponse.text)[1:]
    Pops = pd.DataFrame(columns = json.loads(apiResponse.text)[0], data = formattedResponse)
    
    Pops.columns = variableNames
    if verbose == True:
        #print(Pops)
        print(f'    Fixing Pops Table City Names For {states}')
    Pops['City'] = Pops['City'].str.split(', ', n = 0, expand = True)
    f = lambda x: ' '.join([item for item in x.split() if item not in ['city', 'town', 'township', 'CDP']])
    Pops['City'] = Pops['City'].apply(f)
    if verbose == True:
        #print(Pops)
        print(f'    Fixed Pops Table City Names For {states}')
    
    if verbose == True:
        #print(Pops)
        print(f'Finished Pops Table For {states}\n')

    Jobs = LinkedIn_Monster_Job_Table(states,
                                      job_search,
                                      verbose = verbose)
    Jobs.City = Jobs.City.str.title()
    Jobs = Jobs.replace({'City':replace_dict})
    states_dict = {}
    for stateC, stateN in zip(state_code.split(','), states):
        states_dict[stateC] = stateN
    print(states_dict)
    Pops = Pops.replace({'State':states_dict})
    print(Pops)
    if verbose == True:
        print(f'Combining Jobs and Pops Tables For {states}')
    Final = pd.merge(Pops, Jobs, on = ['City', 'State'], how = 'right')
    if verbose == True:
        print(f'Finished Combining Jobs and Pops Tables For {states}')
    Final['Date'] = pd.to_datetime(date.today()) - pd.to_timedelta(Final['Days Listed'], unit="D")
    if expand == True:
        return Jobs, Pops, Final
    return Final

## Run example with 'Data Science' job string and Arizona, Colorado, Idaho, Montana, Nevada, Oregon, Utah, and Washington listings

In [3]:
df = Census_Place_Jobs_Table('B01001_001E, B01002_001E, B19013_001E, B25105_001E, B25077_001E', 
                        ['City',
                         'Population',
                         'Median Age',
                         'Median Income',
                         'Median Monthly Housing Costs',
                         'Median Home Value',
                         'State',
                         'Place ID'],
                        ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'Oregon', 'Utah', 'Washington'],
                        {'Salt Lake': 'Salt Lake City',
                         'Salt Lake City Metropolitan Area': 'Salt Lake City',
                         'SLC': 'Salt Lake City',
                         'Slc': 'Salt Lake City',
                         'Hill AFB': 'Roy',
                         'Hill Afb': 'Roy',
                         'Denver Metropolitan Area': 'Denver',
                         'Boise': 'Boise City',
                         "Coeur D'Alene": "Coeur d'Alene",
                         "Coeur D Alene": "Coeur d'Alene"},
                        'Data Scientist',
                        year = 2018,
                        verbose = True,
                        expand = False)

#View table
df

Creating Pops Table For ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'Oregon', 'Utah', 'Washington']
    Fixing Pops Table City Names For ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'Oregon', 'Utah', 'Washington']
    Fixed Pops Table City Names For ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'Oregon', 'Utah', 'Washington']
Finished Pops Table For ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'Oregon', 'Utah', 'Washington']

Creating Jobs Table For Arizona
    Finished Monster Table For Arizona
    Finished LinkedIn Table For Arizona
    Finished Indeed Table For Arizona
Finished creating Jobs Table For Arizona 

Creating Jobs Table For Colorado
    Finished Monster Table For Colorado
    Finished LinkedIn Table For Colorado
    Finished Indeed Table For Colorado
Finished creating Jobs Table For Colorado 

Creating Jobs Table For Idaho
    Finished Monster Table For Idaho
    Finished LinkedIn Table For Idaho
    Finished Indeed Table For Idaho
Fini

,City,Population,Median Age,Median Income,Median Monthly Housing Costs,Median Home Value,State,Place ID,Title,Company,Link,Days Listed,Source,Date
0,Wolf Point,2799,31.5,44276,533,101700,Montana,81475,"Medical Technologist / MLT, MT, MLS",McCall and Lee,https://www.linkedin.com/jobs/view/medical-tec...,30,LinkedIn,2020-06-16
1,Cascade,653,54.0,42112,630,131000,Montana,12775,Data Scientist,Fulcrum Worldwide,https://www.linkedin.com/jobs/view/data-scient...,3,LinkedIn,2020-07-13
2,Kalispell,22621,35.9,47362,846,214100,Montana,40075,Health Insurance Agent - Work when & where you...,Assurance,https://job-openings.monster.com/health-insura...,30,Monster,2020-06-16
3,Great Falls,58990,38.6,45620,760,168900,Montana,32800,Health Insurance Agent - Work when & where you...,Assurance,https://job-openings.monster.com/health-insura...,30,Monster,2020-06-16
4,Great Falls,58990,38.6,45620,760,168900,Montana,32800,On-Demand Health Insurance Agent - Think Uber ...,Assurance,https://job-openings.monster.com/on-demand-hea...,30,Monster,2020-06-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,Greater Phoenix Area,NaN,NaN,NaN,NaN,NaN,Arizona,NaN,Senior Data Scientist,Tech Finders,https://www.linkedin.com/jobs/view/senior-data...,3,LinkedIn,2020-07-13
453,Curtin,NaN,NaN,NaN,NaN,NaN,Oregon,NaN,Data Analyst,The Voleon Group,https://www.linkedin.com/jobs/view/data-analys...,30,LinkedIn,2020-06-16
454,Bayview,NaN,NaN,NaN,NaN,NaN,Idaho,NaN,Computer Scientist,United States Department of Defense,https://www.linkedin.com/jobs/view/computer-sc...,90,LinkedIn,2020-04-17
455,Bayview,NaN,NaN,NaN,NaN,NaN,Idaho,NaN,Operations Research,United States Department of Defense,https://www.linkedin.com/jobs/view/operations-...,90,LinkedIn,2020-04-17
